[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/trevortknguyen/nonparametric/blob/master/product_kernels.ipynb)

# Understanding product kernels
A product kernel is a way to do multivariate kernel density estimation where instead of using a covariance matrix as a bandwidth, you take a product of multiple univariate kernels, one for each dimension.

In [ ]:
import scipy.stats
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

## Step 1: Check out ways to simulate multivariate distributions
This code comes from the `scipy` documentation. It allows specifying a full covariance matrix for the spread of the Gaussian. We probably don't need this and can specify the identity matrix.

In [ ]:
x, y = np.mgrid[-1:1:.01, -1:1:.01]
pos = np.empty(x.shape + (2,))
pos[:, :, 0] = x; pos[:, :, 1] = y
rv = scipy.stats.multivariate_normal([0.5, -0.2], [[2.0, 0.3], [0.3, 0.5]])
plt.contourf(x, y, rv.pdf(pos))

In [ ]:
rv = scipy.stats.multivariate_normal([0.5, -0.2])
plt.contourf(x, y, rv.pdf(pos))

We can take a sample from this distribution we just created.

In [ ]:
rv.rvs(500)

## Step 2: